In [1]:
from glob import glob
from tqdm import tqdm
import json

from matplotlib import pyplot as plt

import numpy as np
import pandas as pd

In [2]:
train = pd.read_csv('train_Y.csv') #오프라인 환경에서 진행하였기 때문에 디렉토리 변경
submit = pd.read_csv('sample_submit.csv')

In [3]:
train_user = train['user']
test_user = submit['user']
y_train = train['label'].to_numpy() # numpy 형태로 데이터 형식 변환

In [4]:
total_user = pd.concat([train_user, test_user])
total_user = total_user.sort_values().reset_index(drop=True) #train 과 test에 있는 데이터 합쳐서 재정렬

In [5]:
sleep = sorted(glob('StduentLife_EMA/EMA/response/Sleep/*.json'))
social = sorted(glob('StduentLife_EMA/EMA/response/Social/*.json'))
activity = sorted(glob('StduentLife_EMA/EMA/response/Activity/*.json'))
exercise = sorted(glob('StduentLife_EMA/EMA/response/Exercise/*.json'))
stress = sorted(glob('StduentLife_EMA/EMA/response/Stress/*.json'))

추가진행 사항
======

StudentLife_EMA 내부에 존재하는 EMA reponse 데이터 중 사용하지 않은 Exercise 와 Stress에 대한 데이터 전처리 후 학습 모델에 적용

In [6]:
tmp = list()

for i in range(len(sleep)) :
  for k in range(len(total_user)) : 
    if total_user[k] in sleep[i] :
      tmp.append(sleep[i])

sleep = np.array(tmp)

tmp = list()

for i in range(len(social)) :
  for k in range(len(total_user)) : 
    if total_user[k] in social[i] :
      tmp.append(social[i])

social = np.array(tmp)

tmp = list()

for i in range(len(activity)) :
  for k in range(len(total_user)) : 
    if total_user[k] in activity[i] :
      tmp.append(activity[i])

activity = np.array(tmp)

tmp = list()

for i in range(len(exercise)) :
  for k in range(len(total_user)) : 
    if total_user[k] in exercise[i] :
      tmp.append(exercise[i])

exercise = np.array(tmp) #exercise에 대한 응답을 가져오는 과정

tmp = list()

for i in range(len(stress)) :
  for k in range(len(total_user)) : 
    if total_user[k] in stress[i] :
      tmp.append(stress[i])

stress = np.array(tmp) #stress에 대한 응답을 가져오는 과정

In [7]:
# 아래 설명을 기반으로 Sleep EMA 데이터 추출을 위한 코드를 작성하세요.

################################################################

# 1. json load를 통해서 sleep에 저장된 파일을 하나씩 로드한다.
# 2. 유저의 응답 중 'question_id'가 'hour','rate'인 데이터를 파싱한다.
# 3. 이때, 각 응답은 인덱스 값이므로 해당 인덱스에 대응되는 실제 값으로 변환한다.

# Hint1) 응답결과가 없으면 NaN이 된다. 해당 데이터는 추후 Imputer를 통해 처리한다.

# Hint2) How many hours did you sleep last night?
# [1] -> 3, [2]->3.5, [3]->4.0, [4]->4.5 , ... , [18]->11.5, [19]->12

# Hint3) Sleep_rate
# [1] -> 4, [2]->3, [3]->2, [4]->1

# Hint4) 만약 대응되는 인덱스가 없거나 대응되는 답변이 없다면 해당 데이터는 'NaN'으로 설정한다.

################################################################

# Hint5) 아래 예시코드를 활용하여 작성할 수 있다.

people_hour = list()
people_rate = list()

for sp in tqdm(sleep) : 
    
  person_hour = list()
  person_rate = list()
    
  with open(sp) as json_file:
    sleep_data = json.load(json_file)
    
  for res in sleep_data :
    if 'hour' in res : #hour라는 문자가 sleep_data 내부에 있을 경우 조건문 실행
        index = res['hour'] #해당 응답에서 체크한 인덱스를 불러오는 과정
        answer = (int(index)-1)*0.5 + 3 #위에 있는 hint2를 기반으로 응답 값이 나오도록 설정한 계산식
        person_hour.append(str(answer)) #변환 후 person_hour 리스트에 append 하여 저장
    else:
        person_hour.append('NaN') #관련 응답이 없는 경우 NAN으로 설정

    if 'rate' in res : #res라는 문자가 sleep_data 내부에 있을 경우 조건문 실행
        index = res['rate'] #해당 응답에서 체크한 인덱스를 불러오는 과정
        answer = 5 - int(index) #위에 있는 hint3를 기반으로 응답 값이 나오도록 설정한 계산식
        person_rate.append(str(answer)) #변환 후 person_rate 리스트에 append 하여 저장
    else:
        person_rate.append('NaN') #관련 응답이 없는 경우 NAN으로 설정
        
  people_hour.append(person_hour)
  people_rate.append(person_rate)

100%|████████████████████████████████████████████████████████████████████████████████| 46/46 [00:00<00:00, 1318.04it/s]


In [8]:
# 아래 설명을 기반으로 Social EMA 데이터 추출을 위한 코드를 작성하세요.

################################################################

# 1. json load를 통해서 sleep에 저장된 파일을 하나씩 로드한다.
# 2. 유저의 응답 중 'question_id'가 'number'인 데이터를 파싱한다.
# 3. 이때, 각 응답은 인덱스 값이므로 해당 인덱스에 대응되는 실제 값으로 변환한다.

# Hint1) 응답결과가 없으면 NaN이 된다. 해당 데이터는 추후 Imputer를 통해 처리한다.

# Hint2) Social
# [1]->0, [2]->5, [3]->10, [4]->20, [5]->50, [6]->100

# Hint3) 만약 대응되는 인덱스가 없거나 대응되는 답변이 없다면 해당 데이터는 'NaN'으로 설정한다.

################################################################

# Hint4) 아래 예시코드를 활용하여 작성할 수 있다.

people_contact = list() 

for soc in tqdm(social) :
  person_contact = list()
  with open(soc) as json_file:
    social_data = json.load(json_file)
  for res in social_data : 
    if 'number' in res : #number라는 문자가 social_data 내부에 있을 경우 조건문 실행
        index = res['number'] #해당 응답에서 체크한 인덱스를 불러오는 과정
        if(int(index) == 1):#위에 있는 hint2를 기반으로 응답 값이 나오도록 설정한 조건문들
            answer = 0
        elif(int(index) == 2):
            answer = 5
        elif(int(index) == 3):
            answer = 10
        elif(int(index) == 4):
            answer = 20
        elif(int(index) == 5):
            answer = 50
        else:
            answer = 100 #관련 응답이 없는 경우 NAN으로 설정
        person_contact.append(str(answer))
    else:
        person_contact.append('NaN')


  people_contact.append(person_contact)

100%|████████████████████████████████████████████████████████████████████████████████| 46/46 [00:00<00:00, 1246.80it/s]


In [9]:
# 아래 설명을 기반으로 Activity EMA 데이터 추출을 위한 코드를 작성하세요.

################################################################

# 1. json load를 통해서 sleep에 저장된 파일을 하나씩 로드한다.
# 2. 유저의 응답 중 'question_id'가 'other_relaxing','other_working',
#                                 'relaxing','working'인 데이터를 파싱한다.
# 3. 이때, 각 응답은 인덱스 값이므로 해당 인덱스에 대응되는 실제 값으로 변환한다.

# Hint1) 응답결과가 없으면 NaN이 된다. 해당 데이터는 추후 Imputer를 통해 처리한다.

# Hint2) Activity
# [1]->0, [2]->0.11, [3]->0.26, [4]->0.51, [5]->0.76

# Hint3) 만약 대응되는 인덱스가 없거나 대응되는 답변이 없다면 해당 데이터는 'NaN'으로 설정한다.

################################################################

# Hint4) 아래 예시코드를 활용하여 작성할 수 있다.

people_work_alone = list()  
people_work_other = list()
people_relaxing_alone = list() 
people_relaxing_other = list()

for act in tqdm(activity) :

  person_work_alone = list() 
  person_work_other = list()
  person_relaxing_alone = list() 
  person_relaxing_other = list()

  with open(act) as json_file:
    activity_data = json.load(json_file)

  for res in activity_data :
    
    if 'other_relaxing' in res : #other_relaxing라는 문자가 activity_data 내부에 있을 경우 조건문 실행
        index = res['other_relaxing']  #해당 응답에서 체크한 인덱스를 불러오는 과정
        if(int(index) == 1): #위에 있는 hint2를 기반으로 응답 값이 나오도록 설정한 조건문들
            answer = 0
        elif(int(index) == 2): 
            answer = 0.11
        elif(int(index) == 3):
            answer = 0.26
        elif(int(index) == 4):
            answer = 0.51
        else:
            answer = 0.76
        person_relaxing_other.append(str(answer))
    else:
        person_relaxing_other.append('NaN') #관련 응답이 없는 경우 NAN으로 설정
    ##############################################
    # 해당 부분을 채우시오.
    ##############################################

    if 'other_working' in res :  #other_working라는 문자가 activity_data 내부에 있을 경우 조건문 실행
        index = res['other_working'] #해당 응답에서 체크한 인덱스를 불러오는 과정
        if(int(index) == 1): #위에 있는 hint2를 기반으로 응답 값이 나오도록 설정한 조건문들
            answer = 0
        elif(int(index) == 2):
            answer = 0.11
        elif(int(index) == 3):
            answer = 0.26
        elif(int(index) == 4):
            answer = 0.51
        else:
            answer = 0.76
        person_work_other.append(str(answer))
    else:
        person_work_other.append('NaN') #관련 응답이 없는 경우 NAN으로 설정 
    ##############################################
    # 해당 부분을 채우시오.
    ##############################################

    if 'relaxing' in res : #relaxing라는 문자가 activity_data 내부에 있을 경우 조건문 실행
        index = res['relaxing'] #해당 응답에서 체크한 인덱스를 불러오는 과정
        if(int(index) == 1): #위에 있는 hint2를 기반으로 응답 값이 나오도록 설정한 조건문들
            answer = 0
        elif(int(index) == 2):
            answer = 0.11
        elif(int(index) == 3):
            answer = 0.26
        elif(int(index) == 4):
            answer = 0.51
        else:
            answer = 0.76
        person_relaxing_alone.append(str(answer))
    else:
        person_relaxing_alone.append('NaN') #관련 응답이 없는 경우 NAN으로 설정 
    ##############################################
    # 해당 부분을 채우시오.
    ##############################################

    if 'working' in res : #working라는 문자가 activity_data 내부에 있을 경우 조건문 실행
        index = res['working'] #해당 응답에서 체크한 인덱스를 불러오는 과정
        if(int(index) == 1): #위에 있는 hint2를 기반으로 응답 값이 나오도록 설정한 조건문들
            answer = 0
        elif(int(index) == 2):
            answer = 0.11
        elif(int(index) == 3):
            answer = 0.26
        elif(int(index) == 4):
            answer = 0.51
        else:
            answer = 0.76
        person_work_alone.append(str(answer))
    else:
        person_work_alone.append('NaN') #관련 응답이 없는 경우 NAN으로 설정
    ##############################################
    # 해당 부분을 채우시오.
    ##############################################

  people_relaxing_other.append(person_relaxing_other)
  people_work_other.append(person_work_other)
  people_relaxing_alone.append(person_relaxing_alone)
  people_work_alone.append(person_work_alone)

100%|████████████████████████████████████████████████████████████████████████████████| 46/46 [00:00<00:00, 1487.93it/s]


추가코드1
====
exercise와 관련된 EMA response 불러온 후 전처리하는 과정  

Exercise에 포함되어 있는 EMA 응답 관련 정보

1. Have("Did you do vigorous exercise today (don't include walking) such as run, swim, cycle, play a sport")  
->응답 (Yes) 1 2 (No)  
2. Schedule(If no did you want to but couldn't because of your schedule?)  
->응답 (Yes) 1 2 (No)  
3. exercise("If you exercised how long did you exercise for?")  
->응답 "[1]None, [2]<30 mins, [3]30-60 mins, [4]60-90 mins, [5]>90mins, "  
4. walk("How long did you walk for today?)  
->응답 "[1]None, [2]<30 mins, [3]30-60 mins, [4]60-90 mins, [5]>90mins, "  

In [10]:
people_have = list()
people_schedule = list()
people_exercise = list()
people_walk = list()

for sp in tqdm(exercise) :
    
    person_have = list()
    person_schedule = list()
    person_exercise = list()
    person_walk = list()
    
    with open(sp) as json_file:
        exercise_data = json.load(json_file)
    
    for res in exercise_data :
        if 'have' in res : #have라는 문자가 exercise_data 내부에 있을 경우 조건문 실행
            index = res['have'] #해당 응답에서 체크한 인덱스를 불러오는 과정
            if(index == 'null'): #null 값이 들어있는 경우가 있어서 null 값이 있는 경우 NaN으로 교체
                person_schedule.append('NaN')
            else:
                answer = (int(index)-1) #위에 있는 설명을 기반으로 응답 값이 나오도록 설정한 계산식
                person_schedule.append(str(answer))
        else:
            person_have.append('NaN') #관련 응답이 없는 경우 NAN으로 설정
        
        if 'schedule' in res : #schedule라는 문자가 exercise_data 내부에 있을 경우 조건문 실행
            index = res['schedule'] #해당 응답에서 체크한 인덱스를 불러오는 과정
            if(index == 'null'):
                person_schedule.append('NaN')
            else:
                answer = (int(index)-1) #위에 있는 설명을 기반으로 응답 값이 나오도록 설정한 계산식
                person_schedule.append(str(answer))
        else:
            person_schedule.append('NaN') #관련 응답이 없는 경우 NAN으로 설정
        
        if 'exercise' in res : #exercise라는 문자가 exercise_data 내부에 있을 경우 조건문 실행
            index = res['exercise'] #해당 응답에서 체크한 인덱스를 불러오는 과정
            if(int(index) == 1): #위에 있는 설명을 기반으로 응답 값이 나오도록 설정한 조건식
                answer = 0
            elif(int(index) == 2):
                answer = 30
            elif(int(index) == 3):
                answer = 60
            elif(int(index) == 4):
                answer = 90
            else:
                answer = 120
            person_exercise.append(str(answer))
        else:
            person_exercise.append('NaN') #관련 응답이 없는 경우 NAN으로 설정
            
        if 'walk' in res : #walk라는 문자가 exercise_data 내부에 있을 경우 조건문 실행
            index = res['walk'] #해당 응답에서 체크한 인덱스를 불러오는 과정
            if(int(index) == 1): #위에 있는 설명을 기반으로 응답 값이 나오도록 설정한 조건식
                answer = 0
            elif(int(index) == 2):
                answer = 30
            elif(int(index) == 3):
                answer = 60
            elif(int(index) == 4):
                answer = 90
            else:
                answer = 120
            person_walk.append(str(answer))
        else:
            person_walk.append('NaN') #관련 응답이 없는 경우 NAN으로 설정
        
    people_have.append(person_have)
    people_schedule.append(person_schedule)
    people_exercise.append(person_exercise)
    people_walk.append(person_walk)

100%|████████████████████████████████████████████████████████████████████████████████| 46/46 [00:00<00:00, 1281.18it/s]


추가코드2
====
Stress와 관련된 EMA response 불러온 후 전처리하는 과정

Exercise에 포함되어 있는 EMA 응답 관련 정보

1. level( "Right now, I am...")  
->응답("[1]A little stressed, [2]Definitely stressed, [3]Stressed out, [4]Feeling good, [5]Feeling great, ")

In [11]:
people_level = list()

for sp in tqdm(stress) :
    
    person_level = list()
    
    with open(sp) as json_file:
        stress_data = json.load(json_file)
    
    for res in stress_data : 
        if 'level' in res : #level라는 문자가 exercise_data 내부에 있을 경우 조건문 실행
            index = res['level'] #해당 응답에서 체크한 인덱스를 불러오는 과정
            if(int(index) == 1): #위에 있는 설명을 기반으로 응답 값이 나오도록 설정한 조건식
                answer = 3
            elif(int(index) == 2):
                answer = 4
            elif(int(index) == 3):
                answer = 5
            elif(int(index) == 4):
                answer = 2
            else:
                answer = 1
            person_level.append(str(answer))
        else:
            person_level.append('NaN') #관련 응답이 없는 경우 NAN으로 설정
        
    people_level.append(person_walk)

100%|████████████████████████████████████████████████████████████████████████████████| 46/46 [00:00<00:00, 1048.27it/s]


In [12]:
student_sleep = pd.DataFrame(people_hour).astype('float64')
student_sleep_rate = pd.DataFrame(people_rate).astype('float64')
student_contact = pd.DataFrame(people_contact).astype('float64')
student_working_alone = pd.DataFrame(people_work_alone).astype('float64')
student_working_other = pd.DataFrame(people_work_other).astype('float64')
student_relaxing_alone = pd.DataFrame(people_relaxing_alone).astype('float64')
student_relaxing_other = pd.DataFrame(people_relaxing_other).astype('float64')

student_have = pd.DataFrame(people_have).astype('float64')
student_schedule = pd.DataFrame(people_schedule).astype('float64')
student_exercise = pd.DataFrame(people_exercise).astype('float64')
student_walk = pd.DataFrame(people_walk).astype('float64')
student_stress_level = pd.DataFrame(people_level).astype('float64')

student_sleep['uid'] = total_user
student_sleep_rate['uid'] = total_user
student_contact['uid'] = total_user
student_working_alone['uid'] = total_user
student_working_other['uid'] = total_user
student_relaxing_alone['uid'] = total_user
student_relaxing_other['uid'] = total_user

student_have['uid'] = total_user
student_schedule['uid'] = total_user
student_exercise['uid'] = total_user
student_walk['uid'] = total_user
student_stress_level['uid'] = total_user

student_sleep = student_sleep[(['uid']+list(range(0,student_sleep.shape[1]-1)))]
student_sleep_rate = student_sleep_rate[(['uid']+list(range(0,student_sleep_rate.shape[1]-1)))]
student_contact = student_contact[(['uid']+list(range(0,student_contact.shape[1]-1)))]
student_working_alone = student_working_alone[(['uid']+list(range(0,student_working_alone.shape[1]-1)))]
student_working_other = student_working_other[(['uid']+list(range(0,student_working_other.shape[1]-1)))]
student_relaxing_alone = student_relaxing_alone[(['uid']+list(range(0,student_relaxing_alone.shape[1]-1)))]
student_relaxing_other = student_relaxing_other[(['uid']+list(range(0,student_relaxing_other.shape[1]-1)))]

student_have = student_have[(['uid']+list(range(0,student_have.shape[1]-1)))]
student_schedule = student_schedule[(['uid']+list(range(0,student_schedule.shape[1]-1)))]
student_exercise = student_exercise[(['uid']+list(range(0,student_exercise.shape[1]-1)))]
student_walk = student_walk[(['uid']+list(range(0,student_walk.shape[1]-1)))]
student_stress_level = student_stress_level[(['uid']+list(range(0,student_stress_level.shape[1]-1)))]

In [13]:
# train셋에 대한 feature를 추출하시오.

################################################################

# 1. train_user에 해당하는 데이터만 파싱한다. 
# 2. pandas의 describe 함수를 이용해 feature를 추출한다.
    
################################################################

# Hint 1) 
# A[A['uid'].isin(train_user)]
# Hint 2)
# A.T.describe().to_numpy()

train_features_student_sleep = student_sleep[student_sleep['uid'].isin(train_user)].drop(['uid'],axis=1).T.describe().to_numpy()
train_features_student_sleep_rate = student_sleep_rate[student_sleep_rate['uid'].isin(train_user)].drop(['uid'],axis=1).T.describe().to_numpy()
train_features_student_contact = student_contact[student_contact['uid'].isin(train_user)].drop(['uid'],axis=1).T.describe().to_numpy()
train_features_student_working_alone = student_working_alone[student_working_alone['uid'].isin(train_user)].drop(['uid'],axis=1).T.describe().to_numpy()
train_features_student_working_other = student_working_other[student_working_other['uid'].isin(train_user)].drop(['uid'],axis=1).T.describe().to_numpy()
train_features_student_relaxing_alone = student_relaxing_alone[student_relaxing_alone['uid'].isin(train_user)].drop(['uid'],axis=1).T.describe().to_numpy()
train_features_student_relaxing_other = student_relaxing_other[student_relaxing_other['uid'].isin(train_user)].drop(['uid'],axis=1).T.describe().to_numpy()

train_features_student_have = student_have[student_have['uid'].isin(train_user)].drop(['uid'],axis=1).T.describe().to_numpy()
train_features_student_schedule = student_schedule[student_schedule['uid'].isin(train_user)].drop(['uid'],axis=1).T.describe().to_numpy()
train_features_student_exercise = student_exercise[student_exercise['uid'].isin(train_user)].drop(['uid'],axis=1).T.describe().to_numpy()
train_features_student_walk = student_walk[student_walk['uid'].isin(train_user)].drop(['uid'],axis=1).T.describe().to_numpy()
train_features_student_stress_level = student_stress_level[student_stress_level['uid'].isin(train_user)].drop(['uid'],axis=1).T.describe().to_numpy()

In [14]:
# test셋에 대한 feature를 추출하시오.

################################################################

# 1. test_user에 해당하는 데이터만 파싱한다. 
# 2. pandas의 describe 함수를 이용해 feature를 추출한다.
    
################################################################

# Hint 1) 
# A[A['uid'].isin(test_user)]
# Hint 2)
# A.T.describe().to_numpy()

test_features_student_sleep = student_sleep[student_sleep['uid'].isin(test_user)].drop(['uid'],axis=1).T.describe().to_numpy()
test_features_student_sleep_rate = student_sleep_rate[student_sleep_rate['uid'].isin(test_user)].drop(['uid'],axis=1).T.describe().to_numpy()
test_features_student_contact = student_contact[student_contact['uid'].isin(test_user)].drop(['uid'],axis=1).T.describe().to_numpy()
test_features_student_working_alone = student_working_alone[student_working_alone['uid'].isin(test_user)].drop(['uid'],axis=1).T.describe().to_numpy()
test_features_student_working_other = student_working_other[student_working_other['uid'].isin(test_user)].drop(['uid'],axis=1).T.describe().to_numpy()
test_features_student_relaxing_alone = student_relaxing_alone[student_relaxing_alone['uid'].isin(test_user)].drop(['uid'],axis=1).T.describe().to_numpy()
test_features_student_relaxing_other = student_relaxing_other[student_relaxing_other['uid'].isin(test_user)].drop(['uid'],axis=1).T.describe().to_numpy()

test_features_student_have = student_have[student_have['uid'].isin(train_user)].drop(['uid'],axis=1).T.describe().to_numpy()
test_features_student_schedule = student_schedule[student_schedule['uid'].isin(train_user)].drop(['uid'],axis=1).T.describe().to_numpy()
test_features_student_exercise = student_exercise[student_exercise['uid'].isin(train_user)].drop(['uid'],axis=1).T.describe().to_numpy()
test_features_student_walk = student_walk[student_walk['uid'].isin(train_user)].drop(['uid'],axis=1).T.describe().to_numpy()
test_features_student_stress_level = student_stress_level[student_stress_level['uid'].isin(train_user)].drop(['uid'],axis=1).T.describe().to_numpy()

In [15]:
# Imputer를 이용해 Nan 데이터 평균값으로 채우기

################################################################

# Hint 1) 
# imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
# imp_mean.fit(A)
# A = imp_mean.transform(A)
# B = imp_mean.transform(B)

################################################################
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean')

In [16]:
imputer.fit(train_features_student_sleep) #sleep response에 imputer 적용
train_features_student_sleep = imputer.transform(train_features_student_sleep)
test_features_student_sleep = imputer.transform(test_features_student_sleep)

In [17]:
imputer.fit(train_features_student_sleep_rate) #sleep rate response에 imputer 적용
train_features_student_sleep_rate = imputer.transform(train_features_student_sleep_rate)
test_features_student_sleep_rate = imputer.transform(test_features_student_sleep_rate)

In [18]:
imputer.fit(train_features_student_contact) #student contact에 imputer 적용
train_features_student_contact = imputer.transform(train_features_student_contact)
test_features_student_contact = imputer.transform(test_features_student_contact)

In [19]:
imputer.fit(train_features_student_working_alone) #working alone에 imputer 적용
train_features_student_working_alone = imputer.transform(train_features_student_working_alone)
test_features_student_working_alone = imputer.transform(test_features_student_working_alone)

In [20]:
imputer.fit(train_features_student_working_other) #wroking other에 imputer 적용
train_features_student_working_other = imputer.transform(train_features_student_working_other)
test_features_student_working_other = imputer.transform(test_features_student_working_other)

In [21]:
imputer.fit(train_features_student_relaxing_alone) #relaxing alone에 imputer 적용
train_features_student_relaxing_alone = imputer.transform(train_features_student_relaxing_alone)
test_features_student_relaxing_alone = imputer.transform(test_features_student_relaxing_alone)

In [22]:
imputer.fit(train_features_student_relaxing_other) #relaxing other에 imputer 적용
train_features_student_relaxing_other = imputer.transform(train_features_student_relaxing_other)
test_features_student_relaxing_other = imputer.transform(test_features_student_relaxing_other)

In [23]:
imputer.fit(train_features_student_have) #student have에 imputer 적용
train_features_student_have = imputer.transform(train_features_student_have)
test_features_student_have = imputer.transform(test_features_student_have)

In [24]:
imputer.fit(train_features_student_schedule) #schedule에 imputer 적용
train_features_student_schedule = imputer.transform(train_features_student_schedule)
test_features_student_schedule = imputer.transform(test_features_student_schedule)

In [25]:
imputer.fit(train_features_student_exercise) #exercise에 imputer 적용
train_features_student_exercise = imputer.transform(train_features_student_exercise)
test_features_student_exercise = imputer.transform(test_features_student_exercise)

In [26]:
imputer.fit(train_features_student_walk) #walk에 imputer 적용
train_features_student_walk = imputer.transform(train_features_student_walk)
test_features_student_walk = imputer.transform(test_features_student_walk)

In [27]:
imputer.fit(train_features_student_stress_level) #stress level에 imputer 적용
train_features_student_stress_level = imputer.transform(train_features_student_stress_level)
test_features_student_stress_level = imputer.transform(test_features_student_stress_level)

In [28]:
# Sleep feature만 이용해 우울증을 예측한다. 

#######################################################################

# 1. describe에서 제공하는 통계적 feature 중 count는 제외한 나머지를 사용한다. 

# Hint 1) 아래 feature를 사용하시오

# train
# train_features_student_sleep
# train_features_student_sleep_rate

# test
# test_features_student_sleep
# test_features_student_sleep_rate

# Hint 2) 예시코드

#  tmp_x = list()
#  tmp_x.append(train_features_student_sleep[1:8,i])
#  tmp_x.append(train_features_student_sleep_rate[1:8,i])
#  X_train.append(tmp_x)

#######################################################################

X_train = list()

for i in range(train_features_student_sleep.shape[1]) :
    
    tmp_x = list() #list 선언
    tmp_x.append(train_features_student_sleep[1:8,i]) #sleep feature 저장
    tmp_x.append(train_features_student_sleep_rate[1:8,i]) #sleep rate feature 저장
    X_train.append(tmp_x) #X_train에 append

X_train = np.array(X_train)
X_train = X_train.reshape(train_features_student_sleep.shape[1],-1)

X_test = list()

for i in range(test_features_student_sleep.shape[1]) :
    
    tmp_x = list() #list 선언
    tmp_x.append(test_features_student_sleep[1:8,i]) #sleep feature 저장
    tmp_x.append(test_features_student_sleep_rate[1:8,i]) #sleep rate feature 저장
    X_test.append(tmp_x) #X_train에 append

X_test = np.array(X_test)
X_test = X_test.reshape(test_features_student_sleep.shape[1],-1)

In [29]:
# SVM을 이용해 이진분류 수행하기 

#######################################################################

# Hint 1)
# tuned_parameters = [{'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

#######################################################################
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

parameters = {'C':[1, 10, 100, 1000]} #GridSerachCV에서 변경해서 사용할 만한 데이터 설정
model=GridSearchCV(SVC(kernel = 'linear', class_weight ='balanced'),parameters,cv=5) #model을 GridSearchCV로 설정

model.fit(X_train, y_train) #학습
y_pred = model.predict(X_test) #예측
# Sleep feature를 이용한 분류결과 확인하기 

submit['label'] = y_pred
submit.to_csv("submit_SVC_sleep_features.csv", index=False)

In [30]:
# Activity feature만 이용해 우울증을 예측한다. 

#######################################################################

# 1. describe에서 제공하는 통계적 feature 중 count는 제외한 나머지를 사용한다. 

# Hint 1) 아래 feature를 사용하시오

# train
# train_features_student_contact

# test
# test_features_student_contact
    
#######################################################################

X_train = list()

for i in range(train_features_student_contact.shape[1]) : 
    
    tmp_x = list() #List 선언 
    tmp_x.append(train_features_student_contact[1:8,i]) #contact feature 저장
    X_train.append(tmp_x)

X_train = np.array(X_train)
X_train = X_train.reshape(train_features_student_contact.shape[1],-1)

X_test = list()

for i in range(test_features_student_contact.shape[1]) : 
    
    tmp_x = list() #List 선언 
    tmp_x.append(test_features_student_contact[1:8,i]) #contact feature 저장
    X_test.append(tmp_x)
    
X_test = np.array(X_test)
X_test = X_test.reshape(test_features_student_contact.shape[1],-1)

In [31]:
# SVM을 이용해 이진분류 수행하기 

#######################################################################

# Hint 1)
# tuned_parameters = [{'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

#######################################################################

model.fit(X_train, y_train) #학습
y_pred = model.predict(X_test) #예측
# Activity feature를 이용한 분류결과 확인하기 

submit['label'] = y_pred
submit.to_csv("submit_SVC_activity_features.csv", index=False)

In [45]:
# Social feature만 이용해 우울증을 예측한다. 

#######################################################################

# 1. describe에서 제공하는 통계적 feature 중 count는 제외한 나머지를 사용한다. 

# Hint 1) 아래 feature를 사용하시오

# train
# train_features_student_working_alone
# train_features_student_working_other
# train_features_student_relaxing_alone
# train_features_student_relaxing_other

# test
# test_features_student_working_alone
# test_features_student_working_other
# test_features_student_relaxing_alone
# test_features_student_relaxing_other
    
#######################################################################

X_train = list()

for i in range(train_features_student_sleep.shape[1]) : 
    
    tmp_x = list() #List 선언
    tmp_x.append(train_features_student_working_alone[1:8,i]) #working_alone feature 저장
    tmp_x.append(train_features_student_working_other[1:8,i]) #working_other feature 저장
    tmp_x.append(train_features_student_relaxing_alone[1:8,i]) #relaxing_alone feature 저장
    tmp_x.append(train_features_student_relaxing_other[1:8,i]) #relaxing_other feature 저장
    X_train.append(tmp_x)
    ##############################################
    # 해당 부분을 채우시오.
    ##############################################

X_train = np.array(X_train)
X_train = X_train.reshape(train_features_student_sleep.shape[1],-1)

X_test = list()

for i in range(test_features_student_sleep.shape[1]) :
    
    tmp_x = list() #List 선언
    tmp_x.append(test_features_student_working_alone[1:8,i]) #working_alone feature 저장
    tmp_x.append(test_features_student_working_other[1:8,i]) #working_other feature 저장
    tmp_x.append(test_features_student_relaxing_alone[1:8,i]) #relaxing_alone feature 저장
    tmp_x.append(test_features_student_relaxing_other[1:8,i]) #relaxing_other feature 저장
    X_test.append(tmp_x)
    ##############################################
    # 해당 부분을 채우시오.
    ##############################################

X_test = np.array(X_test)
X_test = X_test.reshape(test_features_student_sleep.shape[1],-1)

In [33]:
# SVM을 이용해 이진분류 수행하기 

#######################################################################

# Hint 1)
# tuned_parameters = [{'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

#######################################################################

model.fit(X_train, y_train) #학습
y_pred = model.predict(X_test) #예측
# Social feature를 이용한 분류결과 확인하기 

submit['label'] = y_pred
submit.to_csv("submit_SVC_social_features.csv", index=False)

In [47]:
# Social feature만 이용해 우울증을 예측한다. 

#######################################################################

# 1. describe에서 제공하는 통계적 feature 중 count는 제외한 나머지를 사용한다. 

# Hint 1) 아래 feature를 사용하시오

# train
# train_features_student_sleep
# train_features_student_sleep_rate
# train_features_student_contact
# train_features_student_working_alone
# train_features_student_working_other
# train_features_student_relaxing_alone
# train_features_student_relaxing_other

# test
# test_features_student_sleep
# test_features_student_sleep_rate
# test_features_student_contact
# test_features_student_working_alone
# test_features_student_working_other
# test_features_student_relaxing_alone
# test_features_student_relaxing_other
    
#######################################################################

X_train = list()

for i in range(train_features_student_working_alone.shape[1]) :
    
    tmp_x = list() #List 선언
    tmp_x.append(train_features_student_sleep[1:8,i]) #sleep feature 저장
    tmp_x.append(train_features_student_sleep_rate[1:8,i]) #sleep_rate feature 저장
    tmp_x.append(train_features_student_contact[1:8,i]) #contact feature 저장
    tmp_x.append(train_features_student_working_alone[1:8,i]) #working_alone feature 저장
    tmp_x.append(train_features_student_working_other[1:8,i]) #working_other feature 저장
    tmp_x.append(train_features_student_relaxing_alone[1:8,i]) #relaxing_alone feature 저장
    tmp_x.append(train_features_student_relaxing_other[1:8,i]) #relaxing_other feature 저장
    X_train.append(tmp_x)

X_train = np.array(X_train)
X_train = X_train.reshape(train_features_student_working_alone.shape[1],-1)

X_test = list()

for i in range(test_features_student_working_alone.shape[1]) :
    
    tmp_x = list() #List 선언
    tmp_x.append(test_features_student_sleep[1:8,i]) #sleep feature 저장
    tmp_x.append(test_features_student_sleep_rate[1:8,i]) #sleep_rate feature 저장
    tmp_x.append(test_features_student_contact[1:8,i]) #contact feature 저장
    tmp_x.append(test_features_student_working_alone[1:8,i]) #working_alone feature 저장
    tmp_x.append(test_features_student_working_other[1:8,i]) #working_other feature 저장
    tmp_x.append(test_features_student_relaxing_alone[1:8,i]) #relaxing_alone feature 저장
    tmp_x.append(test_features_student_relaxing_other[1:8,i]) #relaxing_other feature 저장
    X_test.append(tmp_x) 

X_test = np.array(X_test)
X_test = X_test.reshape(test_features_student_working_alone.shape[1],-1)

In [48]:
np.shape(X_train)

(23, 49)

In [52]:
# SVM을 이용해 이진분류 수행하기 

#######################################################################

# Hint 1)
# tuned_parameters = [{'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

#######################################################################
from imblearn.over_sampling import SMOTE
smote = SMOTE()
X_train_over, y_train_over = smote.fit_resample(X_train, y_train)
model=GridSearchCV(SVC(kernel = 'linear'),parameters,cv=10)
model.fit(X_train_over, y_train_over) #학습
y_pred = model.predict(X_test) #예측
# Sleep+Activity+Social feature를 이용한 분류결과 확인하기 

submit['label'] = y_pred
submit.to_csv("submit_SVC_All_features_test.csv", index=False)

In [51]:
np.shape(X_train_over)

(34, 49)

추가된 데이터를 기반으로 전처리 후 최종 학습 과정
====


In [ ]:
X_train = list()

for i in range(train_features_student_working_alone.shape[1]) :
    
    tmp_x = list()
    tmp_x.append(train_features_student_sleep[1:8,i]) #sleep feature 저장
    tmp_x.append(train_features_student_sleep_rate[1:8,i])  #sleep_rate feature 저장
    tmp_x.append(train_features_student_contact[1:8,i]) #contact feature 저장
    tmp_x.append(train_features_student_working_alone[1:8,i]) #working_alone feature 저장
    tmp_x.append(train_features_student_working_other[1:8,i]) #working_other feature 저장
    tmp_x.append(train_features_student_relaxing_alone[1:8,i]) #relaxing_alone feature 저장
    tmp_x.append(train_features_student_relaxing_other[1:8,i])  #relaxing_other feature 저장
    
    #tmp_x.append(train_features_student_have[1:8,i]) #have feature 저장
    #tmp_x.append(train_features_student_schedule[1:8,i]) #schedule feature 저장
    #tmp_x.append(train_features_student_exercise[1:8,i]) #exercise feature 저장
    #tmp_x.append(train_features_student_walk[1:8,i]) #walk feature 저장
    tmp_x.append(train_features_student_stress_level[1:8,i]) #stress_level feature 저장
    
    X_train.append(tmp_x)
    ##############################################
    # 해당 부분을 채우시오.
    ##############################################

X_train = np.array(X_train)
X_train = X_train.reshape(train_features_student_working_alone.shape[1],-1)

X_test = list()

for i in range(test_features_student_working_alone.shape[1]) :
    
    tmp_x = list() #위의 train에 수행한 것과 같은 과정
    tmp_x.append(test_features_student_sleep[1:8,i])
    tmp_x.append(test_features_student_sleep_rate[1:8,i])
    tmp_x.append(test_features_student_contact[1:8,i])
    tmp_x.append(test_features_student_working_alone[1:8,i])
    tmp_x.append(test_features_student_working_other[1:8,i])
    tmp_x.append(test_features_student_relaxing_alone[1:8,i])
    tmp_x.append(test_features_student_relaxing_other[1:8,i])
    
    #tmp_x.append(test_features_student_have[1:8,i])
    #tmp_x.append(test_features_student_schedule[1:8,i])
    #tmp_x.append(test_features_student_exercise[1:8,i])
    #tmp_x.append(test_features_student_walk[1:8,i])
    tmp_x.append(test_features_student_stress_level[1:8,i])
    
    X_test.append(tmp_x)
    ##############################################
    # 해당 부분을 채우시오.
    ##############################################

X_test = np.array(X_test)
X_test = X_test.reshape(test_features_student_working_alone.shape[1],-1)

In [ ]:
# SVM을 이용해 이진분류 수행하기 

#######################################################################

# Hint 1)
# tuned_parameters = [{'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

#######################################################################

model.fit(X_train, y_train)
y_pred = model.predict(X_test)
# Sleep+Activity+Social feature를 이용한 분류결과 확인하기 

submit['label'] = y_pred
submit.to_csv("submit_SVC_All_features_plus3.csv", index=False)